In [1]:
# load the model first, run the evaluator on the model.

import pandas as pd
import numpy as np
import torch
import utils.print as print_f
import os

from utils.engine import evaluate, get_iou_types
from utils.plot import plot_losses, plot_performance

from models.utils import get_model_size_in_MB
from models.build import create_model_from_setup
from models.setup import ModelSetup
from models.train import TrainingInfo
from utils.save import check_best, end_train, get_data_from_metric_logger
from data.load import get_datasets, get_dataloaders
from IPython.display import clear_output
from utils.eval import get_ap_ar, get_ap_ar_for_train_val
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup, get_coco_eval_params, get_dynamic_loss
from utils.init import reproducibility, clean_memory_get_device
from data.paths import MIMIC_EYE_PATH
# from datetime import datetime
# from models.dynamic_loss import DynamicWeightedLoss
from utils.engine import train_one_epoch, evaluate
from torch import optim
from data.paths import MIMIC_EYE_PATH
from models.load import get_trained_model
# from data.load import seed_worker, get_dataloader_g
from utils.plot import DISEASE_CMAP, get_legend_elements
legend_elements = get_legend_elements(DISEASE_CMAP["solid"])

## Suppress the assignement warning from pandas.r
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
from enum import Enum

class TrainedModels(Enum):
    mobilenet_baseline = "val_lesion-detection_ap_0_1655_test_lesion-detection_ap_0_1648_epoch50_03-15-2023 16-43-54_lesion_dsetection_baseline_mobilenet"  # mobilenet baseline
    mobilenet_with_fix = "val_lesion-detection_ap_0_1918_test_lesion-detection_ap_0_1903_epoch16_03-16-2023 11-34-10_lesion_dsetection_with_fixation_mobilenet"
    resnet18_baseline = "val_lesion-detection_ap_0_1973_test_lesion-detection_ap_0_2010_epoch22_03-16-2023 19-44-55_lesion_dsetection_baseline_resnet"
    resnet18_with_fix = "val_lesion-detection_ap_0_1951_test_lesion-detection_ap_0_2195_epoch12_03-17-2023 00-31-54_lesion_dsetection_with_fixation_resnet"
    densenet161_baseline = "val_lesion-detection_ap_0_1990_test_lesion-detection_ap_0_2085_epoch5_03-17-2023 08-53-33_lesion_dsetection_baseline_densenet161"
    densenet161_with_fix = "val_lesion-detection_ap_0_2120_test_lesion-detection_ap_0_2104_epoch12_03-17-2023 18-36-01_lesion_dsetection_with_fixation_densenet161"
    efficientnet_b5_baseline = "val_lesion-detection_ap_0_1898_test_lesion-detection_ap_0_2055_epoch5_03-17-2023 23-30-57_lesion_dsetection_baseline_efficientnet_b5"
    efficientnet_b5_with_fix = "val_lesion-detection_ap_0_2117_test_lesion-detection_ap_0_2190_epoch8_03-18-2023 12-29-20_lesion_dsetection_with_fixation_efficientnet_b5"
    efficientnet_b0_baseline = "val_lesion-detection_ap_0_1934_test_lesion-detection_ap_0_1858_epoch10_03-18-2023 23-50-47_lesion_dsetection_baseline_efficientnet_b0"
    efficientnet_b0_with_fix = "val_lesion-detection_ap_0_2191_test_lesion-detection_ap_0_2162_epoch10_03-18-2023 19-38-11_lesion_dsetection_with_fixation_efficientnet_b0"
    convnext_base_with_fix = "val_lesion-detection_ap_0_2610_test_lesion-detection_ap_0_2548_epoch22_03-22-2023 02-55-37_lesion_dsetection_with_fixation_convnext_base"
    convnext_base_baseline = "val_lesion-detection_ap_0_2426_test_lesion-detection_ap_0_2325_epoch20_03-22-2023 11-53-53_lesion_dsetection_baseline_convnext_base"
    vgg16_with_fix = "val_lesion-detection_ap_0_2301_test_lesion-detection_ap_0_2186_epoch22_03-20-2023 19-26-02_lesion_dsetection_with_fixation_vgg16"
    vgg16_baseline = "val_lesion-detection_ap_0_2113_test_lesion-detection_ap_0_2068_epoch12_03-21-2023 00-45-24_lesion_dsetection_baseline_vgg16"
    regnet_y_8gf_with_fix = "val_lesion-detection_ap_0_2267_test_lesion-detection_ap_0_2029_epoch12_03-21-2023 11-28-48_lesion_dsetection_with_fixation_regnet_y_8gf"
    regnet_y_8gf_baseline = "val_lesion-detection_ap_0_1883_test_lesion-detection_ap_0_1658_epoch13_03-21-2023 15-22-32_lesion_dsetection_baseline_regnet_y_8gf"

In [3]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [4]:
get_epoch_from_model = lambda select_model: int(([substr for substr in select_model.value.split("_") if "epoch" in substr][0]).replace("epoch", ""))
get_model_name = lambda select_model: str(select_model).split(".")[-1]

In [5]:
select_model = TrainedModels.densenet161_with_fix

In [6]:
model, train_info, optimizer, dynamic_loss_weight = get_trained_model(
        select_model,
        device,
    )

model.to(device)
model.eval()

# cover the long model.
print()

Using pretrained backbone. densenet161
Using pretrained backbone. densenet161
Using SGD as optimizer with lr=0.01



In [7]:
setup = train_info.model_setup
iou_types = get_iou_types(model, setup)

In [8]:
from data.strs import SourceStrs, TaskStrs

dataset_params_dict = {
        "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
        "labels_cols": setup.lesion_label_cols,
        "with_xrays_input": SourceStrs.XRAYS in setup.sources,
        "with_clincal_input": SourceStrs.CLINICAL in setup.sources,
        "with_fixations_input": SourceStrs.FIXATIONS in setup.sources,
        "fixations_mode_input": setup.fiaxtions_mode_input,
        "with_bboxes_label": TaskStrs.LESION_DETECTION in setup.tasks,
        "with_fixations_label": TaskStrs.FIXATION_GENERATION in setup.tasks,
        "fixations_mode_label": setup.fiaxtions_mode_label,
        "with_chexpert_label": TaskStrs.CHEXPERT_CLASSIFICATION in setup.tasks,
        "with_negbio_label": TaskStrs.NEGBIO_CLASSIFICATION in setup.tasks,
        "clinical_numerical_cols": setup.clinical_num,
        "clinical_categorical_cols": setup.clinical_cat,
        "image_size": setup.image_size,
        "image_mean": setup.image_mean,
        "image_std": setup.image_std,
    }

In [9]:
detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
    dataset_params_dict=dataset_params_dict,
)

train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size=setup.batch_size,
)

train_coco = None
train_coco, val_coco, test_coco, eval_dict = get_coco_eval_params(
    source_name=SourceStrs.XRAYS,
    task_name=TaskStrs.LESION_DETECTION,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    detect_eval_dataset=detect_eval_dataset,
    iou_thrs=np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]),
    use_iobb=setup.use_iobb,
    maxDets=[1, 5, 10, 30],
)

In [10]:
test_evaluator, _ = evaluate(
    setup=setup,
    model=model,
    data_loader=test_dataloader,
    device=device,
    params_dict=eval_dict,
    coco=test_coco,
    iou_types=iou_types,
    # score_thres=score_thres,
)

Evaluation:  [ 0/57]  eta: 0:03:31  loss: 0.3649 (0.3649)  lesion-detection_performer-object_detection_loss_classifier: 0.1999 (0.1999)  lesion-detection_performer-object_detection_loss_box_reg: 0.1117 (0.1117)  lesion-detection_performer-object_detection_loss_objectness: 0.0490 (0.0490)  lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.0043 (0.0043)  model_time: 1680086144.0000 (1680086177.7020)  evaluator_time: 0.0000 (0.0000)  time: 3.7020  data: 2.2025  max mem: 1137
Evaluation:  [56/57]  eta: 0:00:02  loss: 0.2141 (0.2027)  lesion-detection_performer-object_detection_loss_classifier: 0.1060 (0.1041)  lesion-detection_performer-object_detection_loss_box_reg: 0.0781 (0.0697)  lesion-detection_performer-object_detection_loss_objectness: 0.0244 (0.0253)  lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.0038 (0.0036)  model_time: 1680086272.0000 (1680086242.7150)  evaluator_time: 0.0000 (0.0000)  time: 2.2195  data: 1.9524  max mem: 1139
Evaluation: Total 

In [1]:
import numpy as np 
from sklearn import metrics
y = np.array([1, 1, 2,2 ])
scores = np.array([0.1, 0.4, 0.35,  0.8])
fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label = 2, )

In [2]:
fpr

array([0. , 0. , 0.5, 0.5, 1. ])

In [3]:
thresholds

array([1.8 , 0.8 , 0.4 , 0.35, 0.1 ])

In [4]:
tpr

array([0. , 0.5, 0.5, 1. , 1. ])